# **The Marginal Contribution of Alpha191 Factors on Non-A-Share Market**

This is a jupyter notebook file for the master thesis titled *The Marginal Contribution of Alpha191 Factors on the Non-A-Share Market*. For clarity, all main code and corresponding explanations have been integrated into this document. This structure allows readers to follow the author’s workflow step by step, facilitating both reproducibility and a deeper understanding of the research.

## Environment Setup

**1. Using conda (Recommended)**

In [2]:
conda env create -f env/environment.yml

3 channel Terms of Service accepted
Channels:
 - defaults
 - conda-forge
Platform: win-64
Solving environment: done

Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Installing pip dependencies: - Ran pip subprocess with arguments:
['C:\\Users\\49282\\miniconda3\\envs\\env_of_alpha\\python.exe', '-m', 'pip', 'install', '-U', '-r', 'd:\\The Marginal Contribution of Alpha191 Factors on Non-A-Share Market - new\\env\\condaenv.pgpj63gj.requirements.txt', '--exists-action=b']
Pip subprocess output:

  Using cached bleach-6.2.0-py3-none-any.whl.metadata (30 kB)


  Using cached certifi-2025.10.5-py3-none-any.whl.metadata (2.5 kB)


  Using cached charset_normalizer-3.4.3-cp312-cp312-win_amd64.whl.metadata (37 kB)


  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)


  Using cached kaggle-1.7.4.5-py3-none-any.whl.metadata (16 kB)


  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)


  Using cached pandas-2.1.3-cp312-cp312-win_am

**2. Using pip**

In [ ]:
pip install -r env/requirements.txt

Before proceeding with the next steps, please ensure your IDE and Jupyter Notebook have been switched to the new environment.

## Dataset Download Guide

This guide provides method to download datasets from Kaggle, especially for the file "usa.csv".

**Step 1: Install Kaggle**

This step should already be completed, if you have finished the **Environment Setup** process. Please check the file path: 
- Windows: C:\Users\\\<username>\
- Linux/Mac: ~/

If you found the ".kaggle" folder, you could skip this step. Otherwise, please manually execute the following code.

In [ ]:
pip install kaggle

**Step 2: Setup API Credentials**

1. Go to Kaggle.com

2. Click on your profile picture → "Settings"

3. Scroll down to "API" section

4. Click "Create New API Token", this downloads "kaggle.json" file

5. Place the file in:

- Windows: C:\Users\\\<username>\\.kaggle\kaggle.json

- Linux/Mac: ~/.kaggle/kaggle.json

**Step 3: Download Dataset**

Please execute the following code to download "usa.csv" dataset. This may take several minutes as the file is approximately 14GB. Please ensure you have sufficient storage space.

In [1]:
from download import *
download_usa_csv()

Dataset URL: https://www.kaggle.com/datasets/jindu4928/usa-csv
usa.csv dataset download completed. Files have been extracted to the 'data' folder.


## Import

In [2]:
import pandas as pd
from class_AlphaCalculator import *
from class_USACalculator import *
from combine_times_files import combine_times_files
from usa_returns import calculateFactorReturns
from class_Alpha191Portfolios import *
from class_UsaPortfolios import *
from class_DSregression import *
from class_Alpha191Portfolios55 import *
from class_UsaPortfolios55 import *

## Calculate Alpha191 Returns

**Read files of s&p500 data to calculate Alpha191 retruns**

In [3]:
CP = pd.read_parquet(r'data\SPXConstituentsPrices.parquet')
CD = pd.read_parquet(r'data\SPXConstituentsDaily.parquet')
CM = pd.read_parquet(r'data\CompleteMapping.parquet')
SP = pd.read_parquet(r'data\SPXPrices.parquet')

**Create an instance object from 'class_AlphaCalculator.py'**

In [4]:
calculator = AlphaCalculator(CP, CD, CM, SP)

**Preprocess data**

In [5]:
calculator.preprocessData()

**Calculate Alpha191 factor value**

Because of the reason of runing time, I split the date into 6 parts:
1. **period0.py**: _'1996-01-02'--'2001-01-02'_
2. **period1.py**: _'2000-01-02'--'2006-01-02'_
3. **period2.py**: _'2005-01-02'--'2011-01-02'_
4. **period3.py**: _'2010-01-02'--'2016-01-02'_
5. **period4.py**: _'2015-01-02'--'2021-01-02'_
6. **period5.py**: _'2020-01-02'--'2023-08-31'_

_find code files above in folder 'period_split'_

And run the 6 parts code together to save time, store results into folder 'period_split' as parquet file.

Because can't run the 6 parts at the same time in jupyter notebook, so I suggest open them and run them manually in different dedicated terminal together. To save time, you can also download the results from kaggle.com. If you want so, please execute the following code.

In [ ]:
download_period_data()

**Merge the 6 files into one parquet file in folder 'factor_value' which named 'alpha_factor'**

In [6]:
combine_times_files()

**Calculate Alpha191 returns and save it into the folder 'factor_returns' as a parquet file named 'alpha_returns'**

In [7]:
alpha_factor = pd.read_parquet(r'factor_value\alpha_factor.parquet')
factor_returns = calculator.calculateFactorReturns(alpha_factor)
factor_returns['period'] = factor_returns['period'].dt.to_timestamp()
factor_returns.to_parquet(r'factor_returns\alpha_returns.parquet', index=False)

Processing alpha_191: 100%|██████████| 236/236 [00:01<00:00, 206.81it/s]


## Calculate Jensen Dataset Returns

**Creat an instance object from 'class_USACalculator.py'**

In [8]:
usa_calculator = USACalculator(
    csv_file=r"data\usa.csv", 
    output_dir='usa_chunks',
    chunk_size=300000
)

**Split Jensen dataset data 'usa.csv' into 14 parquet files and save them into folder 'usa_chunks'**

In [9]:
usa_calculator.csv_to_parquet_chunks()

Created usa_chunks\chunk_1.parquet with 300000 rows
Created usa_chunks\chunk_2.parquet with 300000 rows
Created usa_chunks\chunk_3.parquet with 300000 rows
Created usa_chunks\chunk_4.parquet with 300000 rows
Created usa_chunks\chunk_5.parquet with 300000 rows
Created usa_chunks\chunk_6.parquet with 300000 rows
Created usa_chunks\chunk_7.parquet with 300000 rows
Created usa_chunks\chunk_8.parquet with 300000 rows
Created usa_chunks\chunk_9.parquet with 300000 rows
Created usa_chunks\chunk_10.parquet with 300000 rows
Created usa_chunks\chunk_11.parquet with 300000 rows
Created usa_chunks\chunk_12.parquet with 300000 rows
Created usa_chunks\chunk_13.parquet with 300000 rows
Created final usa_chunks\chunk_14.parquet with 235225 rows


**Filter each chunks and save the filtered parquet files into the same folder**

In [10]:
usa_calculator.filter_chunks()

100%|██████████| 14/14 [04:12<00:00, 18.02s/it]


**Merge all the filtered chunks into one parquet in the folder 'factor_value' named 'usa_factor'**

In [11]:
usa_calculator.merge_filters()

**Calculate usa returns and save it into parquet file in the folder 'factor_returns' named 'usa_returns'**

In [12]:
usa_factor = pd.read_parquet(r'factor_value\usa_factor.parquet')
factor_returns_list = calculateFactorReturns(usa_factor)
factor_returns_list['period'] = factor_returns_list['period'].dt.to_timestamp()
factor_returns_list.to_parquet(r'factor_returns\usa_returns.parquet', index=False)

Processing qmj_safety: 100%|██████████| 1134/1134 [00:02<00:00, 559.49it/s]


## Build 3×2 Bivariate-Sorted Portfolios(Alpha191 part)

**Read file**

In [13]:
alpha_factor = pd.read_parquet(r'factor_value\alpha_factor.parquet')

**Creat an instance object from 'class_Alpha191Portfolios.py'**

In [14]:
alpha_calculator = Alpha191Portfolios(alpha_factor)

**Preprocess 'alpha_factor.parquet' date for building portfolios and following calculation**

In [15]:
alpha_calculator.data_preprocess()

**Build 3×2 Bivariate-Sorted portfolios by sorting prior entire year's characteristics average and market_cap average**

1. Sorting entire year characteristics average and market_cap average to build portfolis using by next year

In [16]:
alpha_calculator.portfolio_group()

Processing alpha_150: 100%|██████████| 19/19 [00:00<00:00, 234.27it/s]


there is a empty list -> i: alpha_150, period: 2010, key: high1
there is a empty list -> i: alpha_150, period: 2017, key: high1
there is a empty list -> i: alpha_150, period: 2018, key: high1
there is a empty list -> i: alpha_150, period: 2019, key: high1
there is a empty list -> i: alpha_150, period: 2020, key: high1
there is a empty list -> i: alpha_150, period: 2021, key: high1
there is a empty list -> i: alpha_150, period: 2022, key: high1


Processing alpha_182: 100%|██████████| 19/19 [00:00<00:00, 189.89it/s]


there is a empty list -> i: alpha_182, period: 2012, key: high1


Processing alpha_191: 100%|██████████| 19/19 [00:00<00:00, 235.46it/s]


2. Calcuate portfolio weighted returns by using prior year's sorting results

In [17]:
alpha_port_ret = alpha_calculator.portfolio_ret()

Processing alpha_191: 100%|██████████| 228/228 [00:00<00:00, 354.30it/s]


**Save results into parquet file in folder 'portfolios' as the name of 'alpha_port_ret'**

In [18]:
alpha_port_ret.to_parquet(r'portfolios\alpha_port_ret.parquet', index=False)

## Build 5×5 Bivariate-Sorted Portfolios(Alpha191 part)

The process is the same as bulding 3×2 Bivariate-Sorted Portfolios(Alpha191 part)

In [19]:
alpha_factor = pd.read_parquet(r'factor_value\alpha_factor.parquet')

In [20]:
alpha55_calculator = Alpha191Portfolios55(alpha_factor)

In [21]:
alpha55_calculator.data_preprocess()

In [22]:
alpha55_calculator.portfolio_group()

Processing alpha_191: 100%|██████████| 19/19 [00:00<00:00, 73.68it/s]


In [23]:
alpha55_port_ret = alpha55_calculator.portfolio_ret()

Processing alpha_191: 100%|██████████| 228/228 [00:01<00:00, 159.83it/s]


In [24]:
alpha55_port_ret.to_parquet(r'portfolios\alpha55_port_ret.parquet', index=False)

## Build 3×2 Bivariate-Sorted Portfolios(Jensen dataset part)

**Read file**

In [25]:
usa_factor = pd.read_parquet(r'factor_value\usa_factor.parquet')

**Creat an instance object from 'class_UsaPortfolios.py'**

In [26]:
usa_calculator = UsaPortfolios(usa_factor)

**Preprocess usa_factor date for building portfolios and following calculation**

In [27]:
usa_calculator.data_preprocess()

**Build 3×2 Bivariate-Sorted portfolios by sorting prior entire year's characteristics average and market_cap average**

1. Sorting entire year characteristics average and market_cap average to build portfolis using by next year

In [28]:
usa_calculator.portfolio_group()

Processing market_equity: 100%|██████████| 19/19 [00:00<00:00, 203.99it/s]


there is a empty list -> i: market_equity, period: 2004, key: low2
there is a empty list -> i: market_equity, period: 2004, key: high1
there is a empty list -> i: market_equity, period: 2005, key: low2
there is a empty list -> i: market_equity, period: 2005, key: high1
there is a empty list -> i: market_equity, period: 2006, key: low2
there is a empty list -> i: market_equity, period: 2006, key: high1
there is a empty list -> i: market_equity, period: 2007, key: low2
there is a empty list -> i: market_equity, period: 2007, key: high1
there is a empty list -> i: market_equity, period: 2008, key: low2
there is a empty list -> i: market_equity, period: 2008, key: high1
there is a empty list -> i: market_equity, period: 2009, key: low2
there is a empty list -> i: market_equity, period: 2009, key: high1
there is a empty list -> i: market_equity, period: 2010, key: low2
there is a empty list -> i: market_equity, period: 2010, key: high1
there is a empty list -> i: market_equity, period: 2011

Processing dolvol_126d: 100%|██████████| 19/19 [00:00<00:00, 192.09it/s]


there is a empty list -> i: dolvol_126d, period: 2004, key: high1
there is a empty list -> i: dolvol_126d, period: 2005, key: high1
there is a empty list -> i: dolvol_126d, period: 2009, key: high1
there is a empty list -> i: dolvol_126d, period: 2010, key: high1
there is a empty list -> i: dolvol_126d, period: 2013, key: high1
there is a empty list -> i: dolvol_126d, period: 2014, key: high1
there is a empty list -> i: dolvol_126d, period: 2015, key: high1
there is a empty list -> i: dolvol_126d, period: 2016, key: high1
there is a empty list -> i: dolvol_126d, period: 2017, key: high1
there is a empty list -> i: dolvol_126d, period: 2018, key: high1
there is a empty list -> i: dolvol_126d, period: 2019, key: high1
there is a empty list -> i: dolvol_126d, period: 2020, key: high1
there is a empty list -> i: dolvol_126d, period: 2021, key: high1
there is a empty list -> i: dolvol_126d, period: 2022, key: high1


Processing ami_126d: 100%|██████████| 19/19 [00:00<00:00, 206.72it/s]


there is a empty list -> i: ami_126d, period: 2004, key: low1
there is a empty list -> i: ami_126d, period: 2004, key: high2
there is a empty list -> i: ami_126d, period: 2005, key: high2
there is a empty list -> i: ami_126d, period: 2006, key: low1
there is a empty list -> i: ami_126d, period: 2006, key: high2
there is a empty list -> i: ami_126d, period: 2007, key: high2
there is a empty list -> i: ami_126d, period: 2008, key: low1
there is a empty list -> i: ami_126d, period: 2008, key: high2
there is a empty list -> i: ami_126d, period: 2009, key: low1
there is a empty list -> i: ami_126d, period: 2009, key: high2
there is a empty list -> i: ami_126d, period: 2010, key: low1
there is a empty list -> i: ami_126d, period: 2010, key: high2
there is a empty list -> i: ami_126d, period: 2011, key: low1
there is a empty list -> i: ami_126d, period: 2011, key: high2
there is a empty list -> i: ami_126d, period: 2012, key: low1
there is a empty list -> i: ami_126d, period: 2012, key: high2

Processing qmj_safety: 100%|██████████| 19/19 [00:00<00:00, 193.43it/s]


2. Calcuate portfolio weighted returns by using prior year's sorting results

In [29]:
usa_port_ret = usa_calculator.portfolio_ret()

Processing qmj_safety: 100%|██████████| 228/228 [00:00<00:00, 285.39it/s]


**Save results into parquet file in folder 'portfolios' as the name of 'usa_port_ret'**

In [30]:
usa_port_ret.to_parquet(r'portfolios\usa_port_ret.parquet', index=False)

## Build 5*5 Bivariate-Sorted Portfolios(Jensen dataset part)

The process is the same as bulding 3×2 Bivariate-Sorted Portfolios(Jensen dataset part)

In [31]:
usa_factor = pd.read_parquet(r'factor_value\usa_factor.parquet')

In [32]:
usa55_calculator = UsaPortfolios55(usa_factor)

In [33]:
usa55_calculator.data_preprocess()

In [34]:
usa55_calculator.portfolio_group()

Processing qmj_safety: 100%|██████████| 19/19 [00:00<00:00, 63.24it/s]


In [35]:
usa55_port_ret = usa55_calculator.portfolio_ret()

Processing qmj_safety: 100%|██████████| 228/228 [00:01<00:00, 191.33it/s]


In [36]:
usa55_port_ret.to_parquet(r'portfolios\usa55_port_ret.parquet', index=False)

## Regression of Double-Selection(DS), Single-Selection(SS), Elastic Net(EN), and Principal Component Analysis(PCA) using 3×2 Bivariate-Sorted Portfolios

**Read file**

In [37]:
alpha_port = pd.read_parquet(r'portfolios\alpha_port_ret.parquet')
usa_port = pd.read_parquet(r'portfolios\usa_port_ret.parquet')
alpha_ret = pd.read_parquet(r'factor_returns\alpha_returns.parquet')
usa_ret = pd.read_parquet(r'factor_returns\usa_returns.parquet')

**Creat an instance object from 'class_DSregression.py'**

In [38]:
DS_calculator = DSregression(alpha_port, usa_port, alpha_ret, usa_ret )

**Preprocess data**

In [39]:
DS_calculator.preprocessData()

**DS regression and get the first LASSO results "I_1", second LASSO reults "I_2"**

In [40]:
I_1, I_2 = DS_calculator.DSregression()

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.996
Model:                            OLS   Adj. R-squared:                  0.995
Method:                 Least Squares   F-statistic:                     1791.
Date:                Mon, 06 Oct 2025   Prob (F-statistic):               0.00
Time:                        12:33:15   Log-Likelihood:                 11740.
No. Observations:                1869   AIC:                        -2.305e+04
Df Residuals:                    1656   BIC:                        -2.187e+04
Df Model:                         212                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -0.0001   2.18e-05  

In [41]:
I_1

{'beta_60m'}

In [42]:
I_2

{'aliq_at',
 'ami_126d',
 'beta_60m',
 'betadown_252d',
 'bev_mev',
 'cash_at',
 'chcsho_12m',
 'corr_1260d',
 'debt_gr3',
 'debt_me',
 'div12m_me',
 'dolvol_126d',
 'dsale_drec',
 'ebit_sale',
 'ebitda_mev',
 'emp_gr1',
 'eqnpo_12m',
 'eqnpo_me',
 'eqpo_me',
 'fcf_me',
 'market_equity',
 'ni_ivol',
 'ni_me',
 'ocf_me',
 'ocfq_saleq_std',
 'op_atl1',
 'prc',
 'prc_highprc_252d',
 'qmj_prof',
 'qmj_safety',
 'rd5_at',
 'ret_1_0',
 'ret_60_12',
 'sale_gr1',
 'sale_gr3',
 'sale_me',
 'saleq_su',
 'seas_11_15na',
 'seas_16_20na',
 'seas_2_5na',
 'tangibility',
 'turnover_126d',
 'z_score',
 'zero_trades_126d',
 'zero_trades_252d'}

**SS regression and get the LASSO results "I"**

In [43]:
I = DS_calculator.SSregression()

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.995
Model:                            OLS   Adj. R-squared:                  0.995
Method:                 Least Squares   F-statistic:                     2189.
Date:                Mon, 06 Oct 2025   Prob (F-statistic):               0.00
Time:                        12:33:53   Log-Likelihood:                 11685.
No. Observations:                1869   AIC:                        -2.303e+04
Df Residuals:                    1700   BIC:                        -2.210e+04
Df Model:                         168                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0001   2.07e-05     -6.438      0.0

In [44]:
I

{'beta_60m'}

**EN regression and get the results "I_3"**

In [45]:
I_3 = DS_calculator.ENregression()

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.996
Model:                            OLS   Adj. R-squared:                  0.995
Method:                 Least Squares   F-statistic:                     2094.
Date:                Mon, 06 Oct 2025   Prob (F-statistic):               0.00
Time:                        12:34:00   Log-Likelihood:                 11709.
No. Observations:                1869   AIC:                        -2.306e+04
Df Residuals:                    1689   BIC:                        -2.206e+04
Df Model:                         179                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -0.0001 

In [46]:
I_3

{'beta_60m',
 'cop_at',
 'debt_gr3',
 'earnings_variability',
 'eqnetis_at',
 'fcf_me',
 'market_equity',
 'netis_at',
 'prc_highprc_252d',
 'resff3_12_1',
 'sale_me',
 'seas_16_20na'}

**PCA regression**

In [47]:
I_4 = DS_calculator.PCAregression()

The number of original control factors: 153
The number of retaining principal components: 2
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.995
Model:                            OLS   Adj. R-squared:                  0.995
Method:                 Least Squares   F-statistic:                     2174.
Date:                Mon, 06 Oct 2025   Prob (F-statistic):               0.00
Time:                        12:34:06   Log-Likelihood:                 11685.
No. Observations:                1869   AIC:                        -2.303e+04
Df Residuals:                    1699   BIC:                        -2.209e+04
Df Model:                         169                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------

## Regression of Double-Selection(DS) using 5×5 Bivariate-Sorted Portfolios

**Read file**

In [48]:
alpha55_port = pd.read_parquet(r'portfolios\alpha55_port_ret.parquet')
usa55_port = pd.read_parquet(r'portfolios\usa55_port_ret.parquet')
alpha_ret = pd.read_parquet(r'factor_returns\alpha_returns.parquet')
usa_ret = pd.read_parquet(r'factor_returns\usa_returns.parquet')

**Creat an instance object from 'class_DSregression.py'**

In [49]:
DS_calculator = DSregression(alpha55_port, usa55_port, alpha_ret, usa_ret )

**Preprocess data**

In [50]:
DS_calculator.preprocessData()

**DS regression and get the first LASSO results "I_1", second LASSO reults "I_2"**

In [51]:
I_1, I_2 = DS_calculator.DSregression()

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.977
Model:                            OLS   Adj. R-squared:                  0.974
Method:                 Least Squares   F-statistic:                     322.0
Date:                Mon, 06 Oct 2025   Prob (F-statistic):               0.00
Time:                        12:35:47   Log-Likelihood:                 10155.
No. Observations:                1864   AIC:                        -1.988e+04
Df Residuals:                    1648   BIC:                        -1.868e+04
Df Model:                         215                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                -9.944e-05 

In [52]:
I_1

{'beta_60m'}

In [53]:
I_2

{'age',
 'aliq_at',
 'aliq_mat',
 'ami_126d',
 'at_turnover',
 'be_gr1a',
 'beta_60m',
 'betabab_1260d',
 'betadown_252d',
 'capx_gr2',
 'cash_at',
 'chcsho_12m',
 'coa_gr1a',
 'cop_atl1',
 'corr_1260d',
 'coskew_21d',
 'cowc_gr1a',
 'debt_gr3',
 'div12m_me',
 'dolvol_126d',
 'dolvol_var_126d',
 'dsale_dinv',
 'dsale_drec',
 'dsale_dsga',
 'earnings_variability',
 'ebit_bev',
 'ebit_sale',
 'ebitda_mev',
 'emp_gr1',
 'eqnetis_at',
 'eqnpo_12m',
 'eqnpo_me',
 'eqpo_me',
 'fcf_me',
 'fnl_gr1a',
 'inv_gr1a',
 'iskew_capm_21d',
 'ival_me',
 'ivol_capm_21d',
 'kz_index',
 'lnoa_gr1a',
 'lti_gr1a',
 'market_equity',
 'mispricing_perf',
 'ncoa_gr1a',
 'netdebt_me',
 'netis_at',
 'nfna_gr1a',
 'ni_ar1',
 'ni_be',
 'ni_ivol',
 'ni_me',
 'niq_at_chg1',
 'niq_be',
 'niq_be_chg1',
 'niq_su',
 'noa_at',
 'noa_gr1a',
 'oaccruals_at',
 'oaccruals_ni',
 'ocf_at',
 'ocf_at_chg1',
 'ocf_me',
 'ocfq_saleq_std',
 'pi_nix',
 'prc',
 'prc_highprc_252d',
 'qmj_growth',
 'qmj_safety',
 'rd_me',
 'resff3_12_1'